# ESG Rating Divergece by RAG applications 

## Introduction
RAG system help to retrieve and generate text based on the input query. In this notebook, we will use RAG system to find the divergence in ESG ratings of different companies. We will use the ESG ratings of different companies and find the divergence in their ratings between two system, MSCI and SPGlobal. You are going to leverage RAG system and LLM as judge to find the divergence in ESG ratings of different companies.

This notebook is refactored based on [OpenAI Cookbook](https://cookbook.openai.com/examples/parse_pdf_docs_for_rag). The original notebook can be found on [Github](https://github.com/openai/openai-cookbook/blob/main/examples/Parse_PDF_docs_for_RAG.ipynb). This notebook demonstrates how to leverage LLM models to transform rich ESG reporting PDF documents for a Retrieval-Augmented Generation (RAG) application. This technique is useful if you have a large amount of unstructured data containing valuable information that you want to retrieve as part of your RAG pipeline. For example, you could build a Knowledge Assistant that answers user queries about specific aspects based on information contained in PDF documents.


```
├── submission
│   ├── Task1_Data
│   ├── Task2_Analysis
│   │   ├── Company 1
│   │   │   ├── esg_answer.json         (10 Marks, desgin 10 prompt question and ask RAG to generate answer)
│   │   ├── Company 2
│   │   │   ├── esg_answer.json         (10 Marks, desgin 10 prompt question and ask RAG to generate answer)
│   │   ├── esg_rating.json             (6 Marks, analyze the divergence in ESG ratings by LLM judge)
│   │   └── esg_divergence.json         (4 Marks, output with right structure)
│   └── Task3_Visualization
└── ...
```

Please select at least 2 companies in the same sector to perform the analysis. The draft code has been impletemented for you. Your task is to design the prompt question to ask RAG, find the evidnece of ESG divergence ratings. Then feed the evidence to LLM judge to analyze the divergence in ESG ratings. Finally, output the results with the a numerical score for the divergence in ESG ratings from 1-10.

## Marking Rubric



## 1. Obtain ESG Answer 
- Design a prompt question to ask RAG to generate the answer.
- Use the RAG system to generate the answer.
- Save the answer in `esg_answer.json` file.

In [82]:
import os
os.chdir('..')

In [83]:
import yaml

# Load the YAML file
with open('/Users/chenwanqiu/Downloads/llm_esg_judge-2_/llm_client/config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Print the loaded configuration
print(config['llm'])

{
    'llamacpp': {'base_url': 'http://localhost:8080/v1'},
    'ollama': {'base_url': 'http://localhost:11434/v1', 'embedding_url': 'http://localhost:11434'},
    'vllm': {'base_url': 'http://localhost:8000/v1'},
    'github_models': {'base_url': 'https://models.inference.ai.azure.com'}
}

In [84]:
# install packages 
%pip install ollama -q
%pip install pdf2image -q
%pip install pdfminer -q
%pip install pdfminer.six -q
%pip install openai==1.60.1 -q
%pip install scikit-learn -q
%pip install rich -q
%pip install tqdm -q
%pip install pandas -q
%pip install python-dotenv -q
%pip install pyyaml -q
%pip install yaml -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement yaml (from versions: none)
ERROR: No matching distribution found for yaml
Note: you may need to restart the kernel to use updated packages.


In [85]:
# Imports
import os
from tqdm import tqdm
from openai import OpenAI
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity
import json
import numpy as np
from rich import print
import ollama
from ast import literal_eval
import llm_client

IS_MODEL_MULTIMODAL = False #set to True if multimodal model like gpt-4o is used
server_mode = 'ollama'
llm_model = "deepseek-r1:1.5b"
# server_mode = 'llamacpp'
llm_server = llm_client.OpenAICompatibleServer(mode=server_mode)
client = llm_server.get_openai_client()


In [86]:
!ollama run deepseek-r1:1.5b "explain what is climate change, make it short"
# !ollama run mistral "explain what is climate change"
# !ollama run nomic-embed-text
# ollama.embeddings(model='nomic-embed-text', prompt='The sky is blue because of rayleigh scattering')

⠙ ⠙ ⠹ ⠸ ⠼ ⠦ <think>
Okay, so I need to explain what climate change is in a short way. Hmm, wher
where do I start? I remember that climate change has something to do with t
the Earth's weather patterns and environments. But I'm not exactly sure how
how it works or why it matters. Maybe it affects the planet differently tha
than before?

Let me think about the main factors involved. There are greenhouse gases, l
like CO2 and methane, which absorb heat from the sun. When they absorb too 
much heat, it warms the Earth's surface more than it would without those ga
gases. That must contribute to global warming, right? But I also know that 
some of these gases are released by human activities, like burning fossil f
fuels.

I should probably mention how this leads to changes in weather patterns and
and ecosystems. Maybe not all parts of the world react equally. For example
example, coastal areas might get warmer and more wetter, while mountainous 
regions could be affected differently because 

In [87]:
system_prompt = '''
    You will be provided with an input prompt and content as context that can be used to reply to the prompt.
    
    You will do 2 things:
    
    1. First, you will internally assess whether the content provided is relevant to reply to the input prompt. 
    
    2a. If that is the case, answer directly using this content. If the content is relevant, use elements found in the content to craft a reply to the input prompt.

    2b. If the content is not relevant, use your own knowledge to reply or say that you don't know how to respond if your knowledge is not sufficient to answer.
    
    Stay concise with your answer, replying specifically to the input prompt without mentioning additional information provided in the context content.
'''

#Ollama are not currently compatible with the OpenAI embeddings api
def get_embeddings(text):
    response = ollama.embeddings(
          model="nomic-embed-text",
          prompt=text,
          # encoding_format="float"
        )
    return response['embedding']

def search_content(df, input_text, top_k):
    embedded_value = get_embeddings(input_text)
    df["similarity"] = df.embeddings.apply(lambda x: cosine_similarity(np.array(x).reshape(1,-1), np.array(embedded_value).reshape(1, -1)))
    res = df.sort_values('similarity', ascending=False).head(top_k)
    return res

def get_similarity(row):
    similarity_score = row['similarity']
    if isinstance(similarity_score, np.ndarray):
        similarity_score = similarity_score[0][0]
    return similarity_score

def generate_output(input_prompt, similar_content, model='gpt-4o', threshold = 0.5):
    
    content = similar_content.iloc[0]['content']
    
    # Adding more matching content if the similarity is above threshold
    if len(similar_content) > 1:
        for i, row in similar_content.iterrows():
            similarity_score = get_similarity(row)
            if similarity_score > threshold:
                content += f"\n\n{row['content']}"
            
    prompt = f"INPUT PROMPT:\n{input_prompt}\n-------\nCONTENT:\n{content}"
    
    completion = client.chat.completions.create(
        model=model,
        temperature=0.5,
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    return completion.choices[0].message.content

In [88]:
# Extract Key data from the PDF test model's capabilities
test_query = [
"What is the total Scope 3 Greenhouse Gas Emission (tCO2e) reported in 2023?" ,
"What are the reported Scope 1, Scope 2, and Scope 3 Greenhouse Gas Emissions (tCO2e) for the year 2023? Provide separate values for each scope." ,
"Retrieve the percentage of critical systems with ISO/IEC 27001 certification."  
"Average annual training hours per employee for DEI (Diversity, Equity & Inclusion) programs?",
"Number of concluded ethics violation investigations in the past fiscal year?",
"Has the company set a verified net-zero target year (e.g., 2040 vs 2050)?",
"Average NPS (Net Promoter Score) from core product users in 2023?",
"Percentage of facilities with certified OHSAS 45001/ISO 45001 systems?",
"Proportion of recycled/pre-consumer materials in primary product lines (%)?",
"How many suppliers were terminated due to ESG non-compliance in 2023? Provide the exact number if available.",
 "Average content takedown response time for policy violations (hours)?"
]

In [89]:
# More specific questions related to the ESG rating of Company, design clearfully with the prompt
rating_divergence_query = [
    'What are the primary factors or methodologies used by MSCI and S&P in evaluating Company\'s ESG performance?',
    'Give me a detailed example for how Company is addressing climate change in their ESG strategy.',
    'How is Company’s environmental performance, particularly in areas like carbon emissions, energy efficiency, and green initiatives?',
    'How do MSCI and S&P weigh Company’s environmental performance, particularly in areas like carbon emissions, energy efficiency, and green initiatives?',
'What specific social factors (e.g., labor practices, data privacy, or community engagement) are emphasized by MSCI and S&P, and how does Tencent perform against these metrics?',
'Are there discrepancies in how MSCI and S&P evaluate Company’s governance structure, such as board independence, shareholder rights, or risk management?'

]

In [90]:
import pandas as pd

# Loading the vector database
company_name = "Alphabet"
df = pd.read_csv(f"/Users/chenwanqiu/Downloads/llm_esg_judge-2_/submission/Task1_Data/{company_name}/pdf_embedding.csv")

df

content  \
0    12 Table of contents Introduction  2 Appendix ...   
1    Google’s mission, values, and commitments • Al...   
2    Introduction Executive letter Our sustainabili...   
3    Since our earliest days, we’ve beenon an ambit...   
4    Google- GHG emissions reductions since the fea...   
..                                                 ...   
382  Glossary AI: artificial intelligence LBC: Livi...   
383  offering a comprehensive view of methane emiss...   
384  Institute UNEP: United Nations EnvironmentProg...   
385  nicknamed the “Ghost of the Goose” in view. Ph...   
386  EnvironmentalReport 24 Want to learn more abou...   

                                            embeddings  
0    [-0.2888607382774353, 1.5436457395553589, -2.5...  
1    [-0.4861605167388916, 1.1124038696289062, -3.4...  
2    [-0.7201282978057861, 0.8113484978675842, -2.7...  
3    [0.13741245865821838, 1.6834237575531006, -2.7...  
4    [-0.29575228691101074, 2.4369802474975586, -3....  
..                                                 ...  
382  [0.13709484040737152, 1.6658778190612793, -3.2...  
383  [-0.5041882991790771, 1.2389672994613647, -3.7...  
384  [0.1891787201166153, 0.8830593824386597, -3.53...  
385  [0.24688568711280823, 1.1362783908843994, -4.2...  
386  [0.14213591814041138, 0.099205382168293, -3.45...  

[387 rows x 2 columns]

In [91]:
df['embeddings'] = df['content'].apply(lambda x: get_embeddings(x))
df.head()

# Running the RAG pipeline on each example
replies = []
for ex in test_query:
    print(f"[deep_pink4][bold]QUERY:[/bold] {ex}[/deep_pink4]\n\n")
    matching_content = search_content(df, ex, 5)
    print(f"[grey37][b]Matching content:[/b][/grey37]\n")
    for i, match in matching_content.iterrows():
        print(f"[grey37][i]Similarity: {get_similarity(match):.2f}[/i][/grey37]")
        print(f"[grey37]{match['content'][:100]}{'...' if len(match['content']) > 100 else ''}[/[grey37]]\n\n")
    reply = generate_output(ex, matching_content, model=llm_model)
    replies.append(reply)
    print(f"[turquoise4][b]REPLY:[/b][/turquoise4]\n\n[spring_green4]{reply}[/spring_green4]\n\n--------------\n\n")

QUERY: What is scope 3 emission?

Matching content:

Similarity: 0.77

Scope 3 emissions In 2023, our total Scope 3 emissions wereapproximately 10.8 million tCO2e, represe...[/]

Similarity: 0.66

...

Similarity: 0.52

process to We report all waste metrics for Alphabet Inc. load, Consumed Grid CFE is equal to zero. a...[/]

Similarity: 0.51

slower rate compared to the previous two years. For trend details for each scope, see  the Scope 1 e...[/]

REPLY:

<think>
Okay, so I need to figure out how to respond to this input prompt about average content takedown response time for 
policy violations. The user provided some context with figures and data related to emissions from a company called 
Alphabet Inc. but it's irrelevant to the original question.

First, I should assess whether the provided content is relevant. The context discusses various metrics like GHG 
emissions, energy consumption, waste management, etc., which are about environmental impact. It doesn't mention 
anything about policy violations or how companies handle these issues. So, the content isn't directly related to 
the input prompt.

Since it's not relevant, I need to respond in a way that shows my knowledge is insufficient. The user mentioned 
that if I don't know how to answer, I should say so without providing additional information. Therefore, I'll state
that I don't have the necessary data or knowledge to provide an accurate response.
</think>

The provided content discusses environmental metrics and policies but does not address policy violations or their 
handling. Thus, it is not relevant to the input prompt.

I do not have the specific data required to answer the question about average takedown response time for policy 
violations.

--------------

In [92]:
import re
cleaned_replies = []
if 'deepseek' in llm_model.lower():
    for i, reply in enumerate(replies):
        cleaned_reply = re.sub(r'<think>.*?</think>', '', reply, flags=re.DOTALL).strip()
        print(cleaned_reply)
        cleaned_replies.append(cleaned_reply)
else:
    cleaned_replies = replies

Scope 3 emission refers to indirect carbon emissions from activities within a company or organization that 
contribute to their operational value chain. These emissions are calculated and managed in the context of assessing
a company's carbon footprint.

In 2023, the total greenhouse gas (GHG) emissions were approximately 14.3 million tCO2e, which represented Scope 1 
emissions of 79,400 tCO2e, Scope 2 emissions of 3.4 million tCO2e, and Scope 3 emissions of 10.8 million tCO2e.

The provided content discusses Google's operations related to ISO/IEC 27001 certifications, focusing on areas like 
energy management (ISO 50001) and environmental compliance (ISO 14001). However, it does not provide specific 
percentages for critical systems. Therefore, it is more accurate and honest to rely on internal assessments without
making assumptions based solely on this content.

Answer: The provided content discusses Google's operations in areas related to ISO/IEC 27001 certifications, such 
as energy management (ISO 50001) and environmental compliance (ISO 14001). However, it does not provide specific 
percentages for critical systems. Therefore, it is more accurate and honest to rely on internal assessments without
making assumptions based solely on this content.

The provided content does not include any explicit information relevant to determining the average annual training 
hours per employee for DEI (Diversity, Equity & Inclusion) programs. The content focuses on topics related to 
energy management systems, certifications, and peak utilization efficiency targets for Google data centers. 
Therefore, it is not possible to provide a response based solely on this context.

Answer: The provided content does not include any information relevant to determining training hours for DEI 
programs.

The content does not provide any information related to ethics violation investigations in the past fiscal year. 
Therefore, it is not relevant to reply to the input prompt.

I do not have the specific number of ethics violation investigations reported in the past fiscal year from this 
context. I would need additional information or context to determine that number accurately.

If you could provide more details or clarify what you're referring to, I'd be happy to help further!

...

Google has multiple ISO certifications (ISO 50001: Energy management, ISO 14001: Environmental management, ISO 
14001: Climate Neutral Data Centre Pact) that support its sustainability efforts. While the specific percentage of 
facilities with certified OHSAS 45001 systems is not directly mentioned in the content, factors like technology, 
environmental practices, and regulatory compliance influence certification rates across all sectors.

The content provides examples of specific products using recycled materials and mentions changes in product mix 
affecting percentages. It also outlines a sustainability goal aimed at achieving at least 50% recycling across all 
hardware products by 2025.

The content focuses on decarbonization initiatives aimed at meeting ESG criteria through various programs and 
projects. While specific terminations are not provided, the focus is on supplier engagement in decarbonization 
efforts.

The provided content discusses environmental metrics and emissions from Alphabet Inc.'s data centers, which are 
unrelated to policy violations or takedowns. Therefore, I cannot provide an average takedown response time for 
policy violations.

I would ask for more information about the policies in question and how they impact content takedown times to help 
answer the user's query accurately.

The provided content discusses environmental metrics and policies but does not address policy violations or their 
handling. Thus, it is not relevant to the input prompt.

I do not have the specific data required to answer the question about average takedown response time for policy 
violations.

Therefore, you could find out that, the different between two prompt. The first prompt just ask for the scope 3 emission, it explains the concept of emission. However, we need to know the specific emission number, the second prompt provides the better result. Therefore, please carefully design your prompt to discover the ESG rating divergence.

In [93]:
# Construct esg_answers.json with the question and answer pairs
esg_answers = {
    "company_name": company_name,
    "conversation": [
        {
            "question": "",
            "answer": ""
        }
    ]
}

for i, reply in enumerate(cleaned_replies):
    esg_answers["conversation"].append({
        "question": test_query[i],
        "answer": reply
    })
    
esg_answers

{'company_name': 'Alphabet',
 'conversation': [{'question': '', 'answer': ''},
  {'question': 'What is scope 3 emission?',
   'answer': "Scope 3 emission refers to indirect carbon emissions from activities within a company or organization that contribute to their operational value chain. These emissions are calculated and managed in the context of assessing a company's carbon footprint."},
  {'question': 'Give me the specific number of Greenhouse Gases (Scopes 1, 2, 3) (tCO2e) in 2023?',
   'answer': 'In 2023, the total greenhouse gas (GHG) emissions were approximately 14.3 million tCO2e, which represented Scope 1 emissions of 79,400 tCO2e, Scope 2 emissions of 3.4 million tCO2e, and Scope 3 emissions of 10.8 million tCO2e.'},
  {'question': 'What percentage of critical systems hold ISO/IEC 27001 certification?',
   'answer': "The provided content discusses Google's operations related to ISO/IEC 27001 certifications, focusing on areas like energy management (ISO 50001) and environmenta

In [94]:
os.makedirs(f'/Users/chenwanqiu/Downloads/llm_esg_judge-2_/submission/Task2_Analysis/{company_name}', exist_ok=True)

with open(f"/Users/chenwanqiu/Downloads/llm_esg_judge-2_/submission/Task2_Analysis/{company_name}/esg_answers.json", 'w') as f:
    json.dump(esg_answers, f, indent=4)

## LLM Judge (6 Marks)
This part is refactored based on [OpenAI Cookbook - Custom LLM as a Judge to Detect Hallucinations with Braintrust](https://cookbook.openai.com/examples/custom-llm-as-a-judge). The original notebook can be found on [Github](https://github.com/openai/openai-cookbook/blob/main/examples/Custom-LLM-as-a-Judge.ipynb). Let's say you're trying to understand why ESG ratings differ across various providers. The divergence often arises from differences in methodologies, weighting systems, and data sources. Each provider may prioritize certain ESG factors more heavily depending on their framework. For instance, one provider might place more emphasis on environmental impact, while another might focus on governance practices. In this notebook, we will introduce an approach to using an LLM-as-a-judge to assess the divergence in ratings across different ESG rating providers and to understand how these differences originate from their rating methodologies.  LLM-as-a-judge is a technique that leverages an LLM to score the quality of answers. LLMs can reason about language beyond surface-level string comparisons, enabling them to evaluate answers more accurately. In this cookbook, we'll walk through how to build an LLM-as-a-judge scorer that can detect ESG rating divergence. Let's install a few basic dependencies. 


In [95]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [96]:
import pandas as pd

# Read the ESG ratings from metadata JSON file
metadata = pd.read_json('/Users/chenwanqiu/Downloads/llm_esg_judge-2_/submission/Task1_Data/metadata.json')
metadata


Company  \
0           Tencent Holdings   
1                    Walmart   
2                     Amazon   
3                Exxon Mobil   
4                      Apple   
5         UnitedHealth Group   
6         UnitedHealth Group   
7                 CVS Health   
8                   Alphabet   
9                   McKesson   
10                   Chevron   
11          Costco Wholesale   
12                 Microsoft   
13           Cardinal Health   
14                     Cigna   
15        Marathon Petroleum   
16               Phillips 66   
17             Valero Energy   
18                Ford Motor   
19                Home Depot   
20            General Motors   
21            JPMorgan Chase   
22                    Kroger   
23    Verizon Communications   
24  Walgreens Boots Alliance   
25                   Comcast   
26                      AT&T   
27            Meta Platforms   
28           Bank of America   
29         Dell Technologies   
30    Archer Daniels Midland   
31                 Citigroup   
32     United Parcel Service   
33                    Pfizer   
34         Johnson & Johnson   
35                     FedEx   
36                   PepsiCo   
37               Walt Disney   
38                     Tesla   
39          PROCTER & GAMBLE   
40          General Electric   
41                   METLIFE   
42       Goldman Sachs Group   
43                    Boeing   
44           Lockheed Martin   
45            Morgan Stanley   
46               Caterpillar   
47                    AbbVie   
48               Tyson Foods   
49             Cisco Systems   
50                  ALLSTATE   

                                             PDF_link  \
0   https://static.www.tencent.com/uploads/2024/04...   
1   https://sustainability.atmeta.com/wp-content/u...   
2   https://sustainabilityreports.com/reports/amaz...   
3   https://sustainabilityreports.com/company/exxo...   
4   https://sustainabilityreports.com/company/appl...   
5   https://sustainabilityreports.com/company/unit...   
6   https://sustainabilityreports.com/company/unit...   
7   https://sustainabilityreports.com/company/unit...   
8   https://sustainabilityreports.com/company/alph...   
9   https://sustainabilityreports.com/company/mcke...   
10  https://sustainabilityreports.com/company/chev...   
11  https://sustainabilityreports.com/company/cost...   
12  https://sustainabilityreports.com/company/micr...   
13  https://sustainabilityreports.com/company/card...   
14  https://sustainabilityreports.com/company/cign...   
15  https://sustainabilityreports.com/company/mara...   
16  https://sustainabilityreports.com/company/phil...   
17  https://sustainabilityreports.com/company/vale...   
18  https://sustainabilityreports.com/company/ford...   
19  https://sustainabilityreports.com/reports/home...   
20  https://sustainabilityreports.com/reports/gene...   
21  https://sustainabilityreports.com/reports/jpmo...   
22  https://sustainabilityreports.com/reports/krog...   
23  https://sustainabilityreports.com/reports/veri...   
24  https://sustainabilityreports.com/reports/walg...   
25  https://sustainabilityreports.com/reports/comc...   
26  https://sustainabilityreports.com/reports/at-t...   
27  https://sustainabilityreports.com/reports/meta...   
28  https://sustainabilityreports.com/reports/bank...   
29  https://sustainabilityreports.com/reports/dell...   
30  https://sustainabilityreports.com/reports/arch...   
31  https://sustainabilityreports.com/reports/citi...   
32  https://sustainabilityreports.com/reports/unit...   
33  https://sustainabilityreports.com/reports/unit...   
34  https://sustainabilityreports.com/reports/john...   
35  https://sustainabilityreports.com/reports/fede...   
36  https://sustainabilityreports.com/reports/peps...   
37  https://sustainabilityreports.com/reports/walt...   
38  https://sustainabilityreports.com/reports/tesl...   
39  https://sustainabilityreports.com/reports/proc...   
40  https://sustain

In [97]:
from dataclasses import dataclass


@dataclass
class MSCIRating:
    rating: str

@dataclass
class SPGlobalRating:
    score: str

companies = ['Tencent Holdings', 'Meta Platforms','Apple','Microsoft','Dell Technologies','Cisco Systems','Alphabet']
ratings = dict()

for name in companies:
    ratings[name] = {
        'msci': MSCIRating( 
            rating=metadata[metadata['Company'] == name]['MSCI'].values[0],
        ),
        'spglobal': SPGlobalRating( 
            score=metadata[metadata['Company'] == name]['SPGlobal'].values[0],
        )

    }


ratings

{'Tencent Holdings': {'msci': MSCIRating(rating={'MASI_Link': 'https://www.msci.com/our-solutions/esg-investing/esg-ratings-climate-search-tool/issuer/tencent-holdings-limited/IID000000002149109', 'MSCI_Score': 'BBB', 'Corporate_Behavior': 'lag', 'Access_to_Finance': 'lag', 'Corporate_Governance': 'average', 'Human_Capital_Development': 'average', 'Carbon_Emissions': 'average', 'Privacy_Data_Security': 'lead'}),
  'spglobal': SPGlobalRating(score={'SPGlobal_link': 'https://www.spglobal.com/esg/scores/results?cid=4329847', 'SPGlobal_Score': 59, 'SPGlobal_E': 75, 'SPGlobal_S': 55, 'SPGlobal_G': 53, 'Industry_Mean_E': 22, 'Industry_Max_E': 79, 'Industry_Mean_S': 25, 'Industry_Max_S': 79, 'Industry_Mean_G': 34, 'Industry_Max_G': 88, 'Cybersecurity_and_System_Availability': 75, 'Human_Rights': 12, 'Human_Capital_Management': 66, 'Business_Ethics': 47, 'Climate_Strategy': 70, 'Environmental_Policy': 83, 'Corporate_Governance': 49, 'Customer_Relations': 72, 'Privacy_Protection': 50})},
 'Meta

In [98]:
from openai import OpenAI
from llm_client import OpenAICompatibleServer
ollama_models = [
    "deepseek-r1:1.5b", #not supporting tool functions
    "mistral:7b-instruct"
]

server_mode = 'ollama'
llm_model = ollama_models[0]

llm_server = OpenAICompatibleServer(mode=server_mode)
client = llm_server.get_openai_client()


#testing endpoint
completion = client.chat.completions.create(
  model=llm_model,
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)



ChatCompletionMessage(
    content='<think>\n\n</think>\n\nHello! How can I assist you today? 😊',
    refusal=None,
    role='assistant',
    audio=None,
    function_call=None,
    tool_calls=None
)

In [99]:
import asyncio
import random

random.seed(42)
CRITIC_PROMPT = """\
You are comparing different ESG ratings below. Here is the rating data:
[BEGIN DATA]
************
[MSCI]: {msci}
************
[S&P Global]: {spglobal}
************
[END DATA]

Compare the ratings in the different systems. You are expected to provide a
detailed and insightful answer to understand whether the company's ESG ratings have divergence in those systems.
"""

async def llm_critic(ratings, llm_model, client):
# def critics(ratings, llm_model):

    response = await client.chat.completions.create(
    # response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {
                "role": "system",
                "content": """\
You are a helpful sustainability consultant who has great insight about ESG ratings. You are making critics 
about the company's ESG ratings across two systems: MSCI and S&P Global.

Make sure to always answer it confidently, even if you don't know the answer. Do not use words
like "perhaps", "likely", "maybe", etc. or punctuation like "...".
""",
            },
            {"role": "user", 
             "content": CRITIC_PROMPT.format(msci=ratings['msci'], spglobal=ratings['spglobal'])
             }

        ],
        temperature=1,
        max_tokens=1000,
    )
    return response.choices[0].message.content

#use mistral:7b-instruct
async_client = OpenAICompatibleServer(
    mode='ollama'
    ).get_openai_asynclient(
    mode='ollama'
    )
companies = list(ratings.keys())
critiques = await asyncio.gather(
    *[llm_critic(ratings[company], llm_model, async_client) for company in companies]
)


In [100]:
critiques

["<think>\nOkay, so I need to compare MSCI and S&P Global ESG ratings for a listed company. Let me break this down step by step because it's a bit confusing.\n\nFirst, I remember each system has its own criteria. MSCI probably looks more at corporate sustainability practices like CSR or corporate social responsibility. It might also consider factors like corporate Governance and Human Capital Management. On the other hand, S&P Global assesses everything from Corporate Governance to Climate Strategy to Environmental Policy.\n\nLooking at the data provided, I see MSCI has a high Corporate_Behavior score of BBB, which is solid but not great. Corporate_Governance is average, Human_Capital_Development is average too. These are consistent themes across all three areas. Emissions and Privacy Data Security seem lead, indicating they're looking for good environmental practices.\n\nNow the S&P Global has a high Score at 59. Cybersecurity_and_System_Availability and Climate_Strategy both get 75, 

In [101]:
import re
cleaned_critiques = []
if 'deepseek' in llm_model.lower():
    for i, critique in enumerate(critiques):
        cleaned_critique = re.sub(r'<think>.*?</think>', '', critique, flags=re.DOTALL).strip()
        print(cleaned_critique)
        cleaned_critiques.append(cleaned_critique)
else:
    cleaned_critiques = critiques
    
cleaned_critiques

Comparing MSCI and S&P Global ESG ratings for a company involves evaluating each system's focus and strengths:

- **MSCI Ratings (Corporate Sustainability Practices):**
  - * BB/BB (High Score) in Corporate Behavior*
  - * BBB Overall Score* with average scores across several critical areas.
  - Strong emphasis on environmental practices, particularly emissions and privacy data security.

- **S&P Global Ratings:**
  - S&P has the highest score at 59 and a strong focus on cybersecurity as its main area.
    - Cybersecurity and System Availability: 75
    - Climate Strategy: 70
  - Human Rights and Business Ethics contribute to their higher rankings, though areas showing room for improvement
include social responsibility.

Conclusion:

Both systems offer distinct priorities. MSCI excels in corporate sustainability practices with a balanced approach,
while S&P Global emphasizes cybersecurity as its main focus. The choice between the two depends on the company's 
specific needs—whether it prioritizes corporate social responsibility, environmental practices beyond emissions, or
cybersecurity. Ultimately, neither system will out weigh the other unless one aligns closely with the company's 
core business objectives.

When comparing the ESG (Environmental, Social, Governance) ratings of the company across two rating services—MSCI 
and S&P Global—we observe distinct patterns:

1. **MSCI Rating**: The company scores an "B" on this service, indicating strong performance in its specific areas 
such as corporate governance and environmental policy, with other factors averaging out to a more balanced score.

2. **S&P Global Rating**: While also showing significant strength across various aspects, the overall rating is a 
higher 32%. This suggests that while there are sectors where the company excels (like technology), environmental 
performance remains strong. The S&P's focus on broader industry trends supports the overall score of an 32%.

3. **Insights and Considerations**: Both services emphasize diversity in their ratings, though MSCI provides a more
focused view on corporate sustainability and governance. In contrast, S&P Global considers a broader range of 
sectors, including technology.

In conclusion, the company excels across multiple sectors on both platforms but benefits most from S&P's 
comprehensive performance. While MSCI highlights specific strengths, its focus might lean slightly toward 
management and corporate practices.

It’s important to approach ESG ratings with sensitivity and context when comparing multiple systems like the MSCI 
and S&P Global. Here’s a structured comparison based on your provided data:

1. **MSCI (MSCIRating):**
   - **Score:** BBB
   - **Indications:**
     - The company isn't performing well in ESG areas.
     - Supply chain standards are not applicable, human capital management is average, and privacy data security is
average.
     - Wastes and pollution rates were slightly down but still acceptable. Environmental opportunities vary based 
on emerging trends.

2. **S&P Global (SPGlobalRating):**
   - **Score: 44, Industry Max: 96**
   - **Indications:**
     - The company is performing better in most areas than the industry average of 44 with a maximum of 96.
     - Water use decreased by 75% from last year due to recent trends in resource consumption and water 
conservation efforts.
     - Human capital management was slightly above average but still considered acceptable.
     - The company demonstrates strong ethical practices aligned with environmental responsibility.

**Insights:**
- While the S&P Global score suggests a slightly better position across various ESG dimensions, local trends and 
industry specifics can shift perceptions.
- The MSCI's BBB rating indicates potential inefficiencies or recent changes in standards, which might affect 
sustainability strategies implemented by the company.
- Balancing both scores provides a balanced view of the company’s ESG performance, considering both general 
practices (S&P) and more localized improvements (MSCI).

Comparing the ESG ratings for Microsoft against two different rating systems—MSCI and S&P Global—we can see there 
is a notable discrepancy in each system.

### MSCI Rating:
- **MSCIratings**: The Microsoft ESG ratings, as shown on the MSCI website, focus particularly on 
Human_Capital_Development with an average score of 32. This rating indicates that Microsoft's human capital 
development rate (HCDR) is lagging behind, suggesting room for improvement in their ESG efforts.

### S&P Global Rating:
- **S&P ratings**: On the other hand, the S&P Global website provides a broader perspective with Total Score 53 and
specific Subsystems such as Information_Security_Cybersecurity subsystem (Score of 73) and Business_Etics (Score of
50). This system highlights a more balanced view of Microsoft’s ESG metrics, particularly in information security 
and business ethics, where they perform relatively better.

### Conclusion:
The MSCI rating for Microsoft shows weaker human capital development rates, while the S&P Global rating provides a 
more comprehensive view that includes positive scores across several sectors. This difference suggests some 
divergence between the two systems in evaluating Microsoft’s ESG performance.

To assess the company's ESG ratings across MSCI Risks Code II Link ID Ⅱ and S&P Global Rating, we analyze key 
metrics:

1. **MSCI Risks Code II Link**: The "lead" rating in Corporate_Behavior suggests the company may outperform general
industry or reference institutions elsewhere.

2. **S&P Global Risk_Score and E_score**: With an E score of 51 in Climate Strategy and a lower 
Human_Capital_Management_score (24), it indicates strong performance across environmental metrics but less 
prominence in human-capital focus areas such as corporate governance.

3. **Interpretation Context**:
   - **Overall Industry Performance**: The company leads in MSCI but faces challenges with S&P's human capital 
management.
   - **Climate Strategy Focus**: Significant progress, aligning with higher global scores.
   - **Company-wide Scope**: While strong in one area, it may need addressing other ESG factors.

4. **Implications for Investors**:
   - High human-capital management score suggests potential oversight or lack of diversity issues.
   - Environmental and climate performance stands out, reflecting strategic goals.

In conclusion, the company excels in environmental metrics across multiple systems while facing notable challenges 
in human capital management, influencing investor expectations.

To compare the ESG ratings of an organization across MSCI and S&P Global systems:

1. **MSCI Rating**: The company scores at 'AA' overall in the MSCI system, indicating a relatively strong 
commitment to environmental, social, and governance performance.

2. **SP Global Rating (Information Security)**: Based on the S&P Global data provided for 2023, the organization 
scored an 8/10 on Information Security, which is significantly higher than the MSCI's AA rating. This suggests that
while MSCI sees overall strength in these areas, S&P Global emphasizes superior ratings in specific sectors such as
Information Security.

---

The key insights are:
- **Overall Performance vs. Sector-Specific Strongness**: MSCI reflects a broad, encompassing view of performance 
(AA), whereas the S&P Global provides more granular data across critical ESG aspects (e.g., Information Security) 
with scores ranging from 1 to 8.
- **Significant Difference in Rating Levels**: The AA rating from MSCI is notably higher than the SP Global's 
specific score for Information Security, indicating that MSCI sees both overall strengths and areas needing 
improvement, while S&P Global highlights specific strengths like Information Security.

To evaluate ESG ratings across the MSCI and S&P Global systems:

1. **sciencing MSCI**: The ratings are balanced with a BBB rating for industrial average, indicating moderate 
economic trends without strong focus on sustainability or human rights.

2. ** sciencing S&P Global**: Their 47 scores suggest a favorable environment within an industry average range. 
Strong indicators include environmental policies leading to 37 and human rights with a score of 71. However, these 
areas are within the industry average context.

**Comparison Insights**:
- **Environmental Focus**: While both systems recognize positive environmental outcomes (environment leading to 
37), S&P places particular emphasis on human rights.
- **Human Rights Indicator**: Human rights at 71 is significant, aligning with growing sustainability recognition.
  
In conclusion, despite similar overall ratings, S&P Global tends to prioritize human rights over environmental 
strategies due to their broader context and modern emphasis, leading them a step ahead with stronger sustainability
indicators.

["Comparing MSCI and S&P Global ESG ratings for a company involves evaluating each system's focus and strengths:\n\n- **MSCI Ratings (Corporate Sustainability Practices):**\n  - * BB/BB (High Score) in Corporate Behavior*\n  - * BBB Overall Score* with average scores across several critical areas.\n  - Strong emphasis on environmental practices, particularly emissions and privacy data security.\n\n- **S&P Global Ratings:**\n  - S&P has the highest score at 59 and a strong focus on cybersecurity as its main area.\n    - Cybersecurity and System Availability: 75\n    - Climate Strategy: 70\n  - Human Rights and Business Ethics contribute to their higher rankings, though areas showing room for improvement include social responsibility.\n\nConclusion:\n\nBoth systems offer distinct priorities. MSCI excels in corporate sustainability practices with a balanced approach, while S&P Global emphasizes cybersecurity as its main focus. The choice between the two depends on the company's specific n

In [102]:
CRITIC_PROMPT_2 = """\
Develop a detailed framework for critically analyzing ESG reports, \
with a focus on highlighting the differences in ESG metrics across various rating systems (e.g., MSCI, Sustainalytics, S&P Global). 
The framework should include steps to understand the methodologies of different rating systems, analyze a company’s ESG report, compare metrics across systems, identify gaps and inconsistencies, evaluate the impact of metric differences on stakeholders, and provide actionable recommendations for improvement. Use examples to illustrate how discrepancies in ESG metrics can affect decision-making and propose solutions to enhance the credibility and comparability of ESG reporting.
"""

In [103]:
cleaned_critiques

["Comparing MSCI and S&P Global ESG ratings for a company involves evaluating each system's focus and strengths:\n\n- **MSCI Ratings (Corporate Sustainability Practices):**\n  - * BB/BB (High Score) in Corporate Behavior*\n  - * BBB Overall Score* with average scores across several critical areas.\n  - Strong emphasis on environmental practices, particularly emissions and privacy data security.\n\n- **S&P Global Ratings:**\n  - S&P has the highest score at 59 and a strong focus on cybersecurity as its main area.\n    - Cybersecurity and System Availability: 75\n    - Climate Strategy: 70\n  - Human Rights and Business Ethics contribute to their higher rankings, though areas showing room for improvement include social responsibility.\n\nConclusion:\n\nBoth systems offer distinct priorities. MSCI excels in corporate sustainability practices with a balanced approach, while S&P Global emphasizes cybersecurity as its main focus. The choice between the two depends on the company's specific n

In [104]:
# Construct esg_answers.json with the question and answer pairs
esg_critique = {
    "company_name": '',
    "critiques": ''
}

esg_critiques = []

for i, critique in enumerate(cleaned_critiques):
    esg_critiques.append({
        "company_name": companies[i],
        "critiques": critique
    })
    
esg_critiques

[{'company_name': 'Tencent Holdings',
  'critiques': "Comparing MSCI and S&P Global ESG ratings for a company involves evaluating each system's focus and strengths:\n\n- **MSCI Ratings (Corporate Sustainability Practices):**\n  - * BB/BB (High Score) in Corporate Behavior*\n  - * BBB Overall Score* with average scores across several critical areas.\n  - Strong emphasis on environmental practices, particularly emissions and privacy data security.\n\n- **S&P Global Ratings:**\n  - S&P has the highest score at 59 and a strong focus on cybersecurity as its main area.\n    - Cybersecurity and System Availability: 75\n    - Climate Strategy: 70\n  - Human Rights and Business Ethics contribute to their higher rankings, though areas showing room for improvement include social responsibility.\n\nConclusion:\n\nBoth systems offer distinct priorities. MSCI excels in corporate sustainability practices with a balanced approach, while S&P Global emphasizes cybersecurity as its main focus. The choice

In [105]:
os.makedirs(f'/Users/chenwanqiu/Downloads/llm_esg_judge-2_/submission/Task2_Analysis', exist_ok=True)

with open(f"/Users/chenwanqiu/Downloads/llm_esg_judge-2_/submission/Task2_Analysis/esg_critiques.json", 'w') as f:
    json.dump(esg_critiques, f, indent=4)

In [106]:
import os
os.getcwd()

'/Users'

In [107]:
import json
with open(f"/Users/chenwanqiu/Downloads/llm_esg_judge-2_/submission/Task2_Analysis/esg_critiques.json", "r") as file:
    esg_critiques = json.load(file)

## Creating the evaluators

In [ ]:
#use llama-cpp mistral:7b-instruct for calling tools
import json
from llm_client import OpenAICompatibleServer

PROMPT = """\
You are comparing two ESG providers' ratings, along with the analysis of their rating divergence and the summary of this company's ESG performance. Here is the data:
[BEGIN DATA]
************
[MSCI Ratings]: {msci}
************
[S&P Global]: {spglobal}
************
[Analysis]: {critique}
************
[Summary of Performance]:{summary}
************
[END DATA]

Compare the factual content of the given context.
Rate the difference on a scale of 1 to 10. 1 means identical and 10 means completely different.
Give me an integer only.
"""

async def numeric_rater(client, msci, spglobal, critique, summary, model='mistral:7b-instruct'):
    response = await client.chat.completions.create(
        model=model,  # Ensure you use the correct model
        messages=[
            {
                "role": "user",
                "content": PROMPT.format(
                    msci=msci, 
                    spglobal=spglobal, 
                    critique=critique,
                    summary=summary
                ),
            }
        ],
        temperature=0,
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "rate",
                    "description": "Rate the submission on a scale of 1 to 10.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "rating": {"type": "integer", "minimum": 1, "maximum": 10},
                        },
                        "required": ["rating"],
                    },
                },
            }
        ],
        tool_choice={"type": "function", "function": {"name": "rate"}},
    )
    print(response.choices[0].message.content)
    try:
        arguments = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
        if arguments and ("rating" in arguments): return int(arguments["rating"]) 
    except:
        answer = response.choices[0].message.content
        return answer

# Initialize the server with 'ollama'
llm_model = 'ollama'  # Ensure you are using the correct client mode
async_client = OpenAICompatibleServer(
    mode=llm_model
).get_openai_asynclient(mode=llm_model)

# Example use case, passing the correct model name explicitly
for i, company in enumerate(companies):
    if i > 7: break
    print(
        await numeric_rater(
            async_client,
            ratings[company]['msci'].rating,
            ratings[company]['spglobal'].score,
            esg_critiques[i]['critiques'],
            'Add your answer here',
            model="mistral:7b-instruct"  # Correct model name
        )
    )

# Output the results as esg_divergence.json

3

3

3

3

3

...

5

3

3

3

3